The goal here is to simulate a realistic GPS signal, including transmitter and receiver clock errors, as well as signal propagation delay based on satellite and receiver positions. The code snippets below show how to set up the simulation parameters, retrieve satellite positions from SP3 data, and compute the necessary delays and clock errors.

We will not include relativistic effects or atmospheric delays in this simplified model.

In [ ]:
import os
import utils
from datetime import datetime, timedelta
import gnss_tools.orbits as orbits
import gnss_tools.time as time
import gnss_tools.coords as coords

In [ ]:
# Create local-data directory for storing downloaded data files
data_dir = os.path.join(os.path.dirname(os.path.dirname(utils.__file__)), "local-data")
os.makedirs(data_dir, exist_ok=True)